# BERTを用いたテキスト分類

このノートブックでは、[BERT](https://arxiv.org/abs/1810.04805)を用いて分類器を構築します。BERTは事前学習済みのNLPのモデルであり、2018年にGoogleによって公開されました。データセットとしては、IMDBレビューデータセットを使います。

なお、学習には時間がかかるので、GPUを使うことを推奨します。

## 準備

### パッケージのインストール

In [1]:
!pip install tensorflow-text==2.6.0 tf-models-official==2.6.0

     |████████████████████████████████| 4.4 MB 2.1 MB/s 
     |████████████████████████████████| 1.8 MB 20.9 MB/s 
     |████████████████████████████████| 1.2 MB 26.8 MB/s 
     |████████████████████████████████| 352 kB 53.8 MB/s 
     |████████████████████████████████| 90 kB 9.7 MB/s 
     |████████████████████████████████| 99 kB 8.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 37.1 MB 46 kB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 
     |████████████████████████████████| 211 kB 46.9 MB/s 
     |████████████████████████████████| 636 kB 42.1 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=498f6850638c76e1c107a5faeaac66a385c654b2a1cabd1f4e3f641582403c39
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=717ebbaab75b88dc12

### インポート

In [2]:
import os
import re
import string

import numpy as np
import tensorflow as tf
import tensorflow_text as text
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from official.nlp import optimization

### データセットの読み込み

In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'train[80%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteX508WW/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteX508WW/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## 前処理

前処理としては、以下の3つを行います。

- 小文字化
- HTMLタグの除去（`<br />`タグ）
- 句読点の除去

In [4]:
def preprocessing(input_data, label):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    cleaned_html = tf.strings.regex_replace(
        stripped_html,
        '[%s]' % re.escape(string.punctuation),
        ''
    )
    return cleaned_html, label

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_data.batch(32).map(preprocessing).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_data.batch(32).map(preprocessing).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_data.batch(32).map(preprocessing).cache().prefetch(buffer_size=AUTOTUNE)

## モデルの構築

今回は、[TensorFlow Hub](https://www.tensorflow.org/hub)を用いて、BERTを使ったモデルを構築します。TensorFlow Hubは、学習済みの機械学習モデルのリポジトリです。ここには、BERTを含む多数のモデルが公開されており、ファインチューニングすることで、素早くモデルを構築できます。BERT以外にも、以下のようなモデルが公開されています。

- ALBERT
- Electra
- Universal Sentence Encoder

それでは、TensorFlow Hubを使ってみましょう。

### 前処理モデル

テキストは、BERTへ入力される前に、数値トークンIDに変換される必要があります。TensorFlow Hubは、BERTモデルに対応する前処理モデルを提供しており、それを使うことで、テキストを変換できます。したがって、前処理のために長々とコードを書く必要はありません。以下のように、前処理モデルを指定して読み込むだけです。

In [12]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

前処理モデルの出力を確認してみましょう。

In [13]:
text_test = ['this is such an amazing movie!']
text_preprocessed = preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


ご覧のとおり、前処理モデルは以下の3つの出力をします。

- input_words_id: 入力系列のトークンID
- input_mask: パディングされたトークンには0、それ以外は1
- input_type_ids: 入力セグメントのインデックス。複数の文を入力する場合に関係する。

その他、入力が128トークンに切り詰められていることがわかります。ちなみに、トークン数はオプション引数でカスタマイズできます。詳細は、[前処理モデルのドキュメント](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3)をご覧ください。

### BERTモデル

モデルを構築する前に、BERTモデルの出力を確認してみましょう。

In [16]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [18]:
bert_results = bert_model(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.9216989  -0.39353472 -0.5393176   0.682563    0.43848526 -0.14021198
  0.8774715   0.26043355 -0.63113034 -0.9999658  -0.26320082  0.8510534 ]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[ 0.19451515  0.25141722  0.19075063 ... -0.24845128  0.38568568
   0.1329099 ]
 [-0.5947862  -0.39420295  0.25245643 ... -0.769468    1.1564158
   0.32475588]
 [ 0.00641477 -0.15766507  0.5461029  ... -0.17451143  0.60289675
   0.42672214]
 ...
 [ 0.21948312 -0.20927148  0.5386829  ...  0.24693674  0.18250933
  -0.4442711 ]
 [ 0.01080263 -0.44553217  0.35990965 ...  0.31722867  0.2356279
  -0.63070595]
 [ 0.29321143 -0.10581905  0.61147535 ...  0.2074582   0.14494652
  -0.35353374]]


`pooled_output`と`sequence_output`の説明は以下の通りです。

- pooled_output: 入力全体を表しているベクトルです。レビュー文全体の埋め込みと考えられます。今回のモデルの場合、形は`[batch_size, 768]`になります。上の例では入力は1つだけなので`[1, 768]`になります。
- sequence_output: 各入力トークンを表すベクトルです。各トークンの文脈を考慮した埋め込みと考えられます。形は、`[batch_size, seq_length, 768]`です。

今回は、レビューを分類すればいいので、`pooled_output`を使います。

### モデルの定義

In [8]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess)
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True)
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    return tf.keras.Model(text_input, net)

## モデルの学習

In [10]:
model = build_classifier_model()
epochs = 2
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw'
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['acc']
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
)

Epoch 1/5
625/625 [==============================] - 1211s 2s/step - loss: 0.3985 - acc: 0.8094 - val_loss: 0.3096 - val_acc: 0.8796
Epoch 2/5
625/625 [==============================] - 1192s 2s/step - loss: 0.2145 - acc: 0.9173 - val_loss: 0.3081 - val_acc: 0.8858
Epoch 3/5
625/625 [==============================] - 1190s 2s/step - loss: 0.1165 - acc: 0.9620 - val_loss: 0.5111 - val_acc: 0.8886
Epoch 4/5
625/625 [==============================] - 1181s 2s/step - loss: 0.0594 - acc: 0.9828 - val_loss: 0.5525 - val_acc: 0.8892
Epoch 5/5
625/625 [==============================] - 1184s 2s/step - loss: 0.0269 - acc: 0.9936 - val_loss: 0.5932 - val_acc: 0.8898


In [11]:
loss, accuracy = model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

782/782 [==============================] - 489s 625ms/step - loss: 0.5735 - acc: 0.8926
Loss: 0.5735048651695251
Accuracy: 0.8925999999046326
